This is a simple LLM / RAG chat model with chat history saved

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

# Set environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
# Set up model and embedding
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
model = ChatOpenAI(model="gpt-3.5-turbo")
embedding = OpenAIEmbeddings()

In [3]:
# Set up basic chroma vectorstore
from langchain_community.vectorstores import Chroma
import document_handler

# https://python.langchain.com/docs/integrations/vectorstores/chroma

chroma_collection_name = "LangChainCollection"

import chromadb
new_client = chromadb.EphemeralClient()

vectorstore_initialize = Chroma.from_documents(
    document_handler.processed_texts,
    embedding=embedding,
    collection_name=chroma_collection_name,
    client=new_client,
)

vectorstore = Chroma(
    client=new_client,
    collection_name=chroma_collection_name,
    embedding_function=embedding,
)
retriever = vectorstore.as_retriever()

In [7]:
docs = vectorstore_initialize.similarity_search("What is Chocolate?")
print(docs)
docs = vectorstore.similarity_search("What is Chocolate?")
print(docs)

[Document(page_content='Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffl

In [4]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import tool
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [38]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="output", input_key="question"
)

In [6]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Chat History: {chat_history}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [20]:
# Prompt

system_message_template = (
    "You are a helpful assistant who helps answer questions. Answer only the facts based on the context. "
    "Your goal is to provide accurate and relevant answers based on the facts in the provided context. "
    "Make sure to reference the above source documents appropriately and avoid making assumptions or adding personal opinions. "
    "Emphasize the use of facts from the provided source documents. "
    "Instruct the model to use source name for each fact used in the response. "
    "Avoid generating speculative or generalized information. "
    "Use square brackets to reference the source, e.g. [info1.txt]. "
    "Do not combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\n"
    "Here is how you should answer every question:\n"
        "-Look for relevant information in the above source documents to answer the question.\n"
        "-If the source document does not include the exact answer, please respond with relevant information from the data in the response along with citation. You must include a citation to each document referenced.\n"
        "-If you cannot find answer in below sources, respond with I am not sure. Do not provide personal opinions or assumptions and do not include citations.\n"
        "-If you use any information in the context, include the index(starts at 1) of the statement as citation in your answer\n"
    "At the end of your response:\n" 
    "1. Add key words from the paragraphs. \n"
    "2. Suggest a further question that can be answered by the paragraphs provided. \n"
    "3. Create a source list of source name, author name, and a link for each document you cited.\n"
    "{context}"

)

MEMORY_KEY = "chat_history"

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message_template),
        # MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("human", "{question}"),
        # MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


In [48]:
# Set llm chain
from langchain_core.messages import get_buffer_string

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser()
}

chain = (
    loaded_memory
    | standalone_question
    | {"context": itemgetter("standalone_question") | retriever | format_docs, "question": lambda x: x["standalone_question"]}
    | final_prompt
    | model
    | StrOutputParser()
)

In [47]:
import pandas as pd

df = pd.read_csv('./test_data/Questions.csv', delimiter=',')
tuples = [tuple(x) for x in df.values]
dicts = df.to_dict('records')

print(dicts)

questions = list(map(lambda x : x['Question'], dicts))
print(questions)

[{'Question': 'What is Chocolate?', 'Follow up': 'No'}, {'Question': 'Is that a word?', 'Follow up': 'Yes'}, {'Question': 'Write the following words and then continue: g\\thumb|350', 'Follow up': 'No'}, {'Question': 'What is iPhone?', 'Follow up': 'No'}, {'Question': 'What is Task Decomposition?', 'Follow up': ' No'}]
['What is Chocolate?', 'Is that a word?', 'Write the following words and then continue: g\\thumb|350', 'What is iPhone?', 'What is Task Decomposition?']


In [49]:
for dict in dicts:
        question = dict['Question']
        follow_up = dict['Follow up']
        if follow_up == "No":
                memory.clear()
        print(memory.load_memory_variables({}))
        llm_response = chain.invoke({"question": question})
        print(llm_response)
        memory.save_context({"question":question}, {"output":llm_response})

memory.clear()

{'history': []}


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-hptLc***************************************yQUo. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}